**Abdullah Ash Saki**  
Enabling Technologies Researcher @ IBM Quantum  
saki@ibm.com

**Pedro Rivero**  
Technical Lead @ IBM Quantum  
pedro.rivero@ibm.com

# Pauli Twirling

## What is twirling?
```
MISSING SCIENTIFIC/THEORETICAL EXPLANATION
```
Let us begin by introducing an auxiliary class `PauliTwirl`, to represent a single Pauli twirl:

In [1]:
from numpy import exp, ndarray
from numpy.typing import ArrayLike
from qiskit.circuit import QuantumRegister
from qiskit.circuit.library import PauliGate
from qiskit.dagcircuit import DAGCircuit, DAGOpNode


class PauliTwirl:
    """Pauli twirl.

    This class holds information about a Pauli twirl, independently
    of what operation it is later applied to. Therefore, applying
    the represented twirl to an arbitrary operation has no guaranty
    of preserving such operation's original action.

    Args:
        pre: Pauli gate to apply before the twirled operation.
        post: Pauli gate to apply after the twirled operation.
        phase: global phase induced by the twirling.
    """

    def __init__(self, pre: str, post: str, phase: float = 0.0) -> None:
        self.pre = PauliGate(pre)
        self.post = PauliGate(post)
        self.phase = float(phase)
        if self.pre.num_qubits != self.post.num_qubits:
            raise ValueError(
                "Twirling pre and post operations don't apply to the same number of qubits."
            )

    @property
    def num_qubits(self) -> int:
        """Number of qubits that the twirl applies to."""
        return self.pre.num_qubits

    def apply_to_node(self, node: DAGOpNode) -> DAGCircuit:
        """Apply twirl to input DAG operation node."""
        dag = DAGCircuit()
        qubits = QuantumRegister(self.num_qubits)
        dag.add_qreg(qubits)
        dag.apply_operation_back(self.pre, qubits)
        dag.apply_operation_back(node.op, qubits)
        dag.apply_operation_back(self.post, qubits)
        dag.global_phase += self.phase
        return dag

    def apply_to_unitary(self, unitary: ArrayLike) -> ndarray:
        """Apply twirl to input unitary."""
        pre = self.pre.to_matrix()
        post = self.post.to_matrix()
        phase_factor = exp(1j * self.phase)
        return (post @ unitary @ pre) * phase_factor


## Operation-preserving twirls
```
MISSING SCIENTIFIC/THEORETICAL EXPLANATION
```
Next we will create a helper function `generate_pauli_twirls` to compute all operation-preserving twirls for a given unitary matrix numerically:

In [2]:
from collections.abc import Iterator
from itertools import product
from numpy import allclose, angle, eye, isclose, ndarray


def generate_pauli_twirls(unitary: ndarray) -> Iterator[PauliTwirl]:
    """Generate operation-preserving Pauli twirls for input unitary.

    Args:
        unitary: the unitary to compute twirls for.

    Yields:
        Twirls preserving the unitary operation. Qubit order is given by the input.
    """
    dimension = unitary.shape[0]
    num_qubits = dimension.bit_length() - 1  # Note: dimension == 2**num_qubits
    n_qubit_paulis = ("".join(pauli) for pauli in product("IXYZ", repeat=num_qubits))
    for pre, post in product(n_qubit_paulis, repeat=2):
        twirl = PauliTwirl(pre, post, phase=0.0)
        twirled = twirl.apply_to_unitary(unitary)
        check = twirled.conj().T @ unitary
        phase_factor = check[0, 0]
        if not isclose(phase_factor, 0) and allclose(check / phase_factor, eye(dimension)):
            yield PauliTwirl(pre=pre, post=post, phase=angle(phase_factor))


## Random twirling
```
MISSING SCIENTIFIC/THEORETICAL EXPLANATION
```
Finally, with these tools, we can create a simple [transpiler pass](https://docs.quantum.ibm.com/transpile/custom-transpiler-pass#transpiler-passes) `TwoQubitPauliTwirlPass` to apply Pauli twirling to an input `DAGCircuit`:

In [3]:
from numpy.random import default_rng
from qiskit.circuit import Operation, Gate
from qiskit.dagcircuit import DAGCircuit
from qiskit.transpiler import TransformationPass


class TwoQubitPauliTwirlPass(TransformationPass):
    """Pauli twirl two-qubit gates in input circuit randomly.

    Both non-unitary and parametrized gates are not supported and will be skipped.

    Args:
        seed: seed for random number generator.
    """

    def __init__(self, *, seed: int | None = None):
        super().__init__()
        self._rng = default_rng(seed)

    def run(self, dag: DAGCircuit):
        """Pauli twirl target gates randomly for input DAGCircuit inplace."""
        target_nodes = (node for node in dag.op_nodes() if self._is_target_op(node.op))
        for node in target_nodes:
            twirl = self._get_random_twirl(node.op)
            twirl_dag = twirl.apply_to_node(node)
            dag.substitute_node_with_dag(node, twirl_dag)
        return dag

    def _is_target_op(self, op: Operation) -> bool:
        """Check whether operation should be twirled or not."""
        if op.num_qubits != 2:
            return False  # Note: Only twirl two-qubit gates
        if not isinstance(op, Gate):
            return False  # Note: Skip non-gate nodes (e.g. barriers, measurements)
        if op.is_parameterized():
            return False  # Note: Skip parametrized gates
        return True

    def _get_random_twirl(self, gate: Gate) -> PauliTwirl:
        """Get random twirl for the input gate."""
        twirls = generate_pauli_twirls(gate.to_matrix())
        return self._rng.choice(list(twirls))


## Usage
Using Qiskit's `PassManager` we can now run a simple example:

In [4]:
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager

circuit = QuantumCircuit(2)
circuit.cx(0, 1)

pass_manager = PassManager(TwoQubitPauliTwirlPass(seed=0))
twirled_circuit = pass_manager.run(circuit)

print(twirled_circuit)

global phase: π
     ┌────────────┐     ┌────────────┐
q_0: ┤0           ├──■──┤0           ├
     │  Pauli(ZX) │┌─┴─┐│  Pauli(YY) │
q_1: ┤1           ├┤ X ├┤1           ├
     └────────────┘└───┘└────────────┘


Which can be further decomposed into single-qubit paulis:

In [5]:
print(twirled_circuit.decompose("pauli"))

global phase: π
     ┌───┐     ┌───┐
q_0: ┤ X ├──■──┤ Y ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ Z ├┤ X ├┤ Y ├
     └───┘└───┘└───┘


And we can see how the unitary is preserved:

In [6]:
from numpy import isclose, pi
from qiskit.circuit.library import CXGate

twirl = PauliTwirl("ZX", "YY", phase=pi)

cx_unitary = CXGate().to_matrix()
twirled_unitary = twirl.apply_to_unitary(cx_unitary)

print("Original: \n", cx_unitary.real)
print("Twirled: \n", twirled_unitary.real)

Original: 
 [[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]
Twirled: 
 [[ 1. -0. -0. -0.]
 [-0. -0. -0.  1.]
 [-0. -0.  1. -0.]
 [-0.  1. -0. -0.]]


In [7]:
print("Mask (isclose): \n", isclose(twirled_unitary, cx_unitary))

Mask (isclose): 
 [[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


---

## References
1. Wallman et al., _Noise tailoring for scalable quantum computation via randomized compiling_, [Phys. Rev. A 94, 052325](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.052325)
2. Minev, _A tutorial on tailoring quantum noise - Twirling 101_, [Online](https://www.zlatko-minev.com/blog/twirling)
3. ...